In [1]:
import sys
sys.path.append('/home/haozhe/Desktop/RoboVerse')


import numpy as np
import torch
import trimesh
import transforms3d
from scripts.real2sim.kinematic.hand_model_lit import HandModelURDFLite

from pathlib import Path
import yaml

HAND_URDF_PATH=Path('/home/haozhe/Desktop/RoboVerse/_data/allegro/allegro_hand_description_dexgrasp/allegro_hand_description_right.urdf')

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# # 读取 YAML 文件
# with open("/home/sichengh/25spring/curobo/src/curobo/content/configs/robot/spheres/franka_allegro_right.yml", "r") as file:
#     collision_spheres = yaml.safe_load(file)['collision_spheres']

# collision_spheres = {k: v for k,
#                      v in collision_spheres.items() if not k.startswith("panda_link")}

In [ ]:
import trimesh
import os
file_path='scripts/real2sim/engine_util/direct_mapping/allegro_hand_description_dexgrasp/meshes/base_link.STL'
link_mesh=trimesh.load_mesh(file_obj=trimesh.util.wrap_as_stream(file_path), file_type='stl', process=False,force='mesh')

In [8]:
def normalize_j0_joints(arr, factor=1.5):

    """
    Multiply the j0 values (indices 0, 4, 8, 12) in the input array by `factor`.

    The expected joint order is:
      0: ffj0,  1: ffj1,  2: ffj2,  3: ffj3,
      4: mfj0,  5: mfj1,  6: mfj2,  7: mfj3,
      8: rfj0,  9: rfj1, 10: rfj2, 11: rfj3,
     12: thj0, 13: thj1, 14: thj2, 15: thj3
    """
    # Indices corresponding to *j0 in your ordering
    j0_indices = [1, 5, 9]
    # Multiply those indices by factor
    arr[j0_indices] *= factor
    return arr


def re_direct_j2_thj(arr):
    """
    reset the orientation of thj2
    """
    arr[14] = arr[14]*-1
    return arr



def norm_th0(arr,scale=1.5):
    """
    normalize thj0
    """
    arr[12] = arr[12]*scale
    return arr

In [18]:

# Initialize the hand model
hand_model = HandModelURDFLite(
    urdf_path=HAND_URDF_PATH,
    device=device,
    # collision_spheres=collision_spheres,
)

# Hardcoded translation and rotation
translation = [0.05, 0.1, 0.15]  # x, y, z in meters
rotation_euler = [0.1, -0.2, 0.3]  # roll, pitch, yaw in radians

# Convert Euler angles to rotation matrix
rotation_matrix = transforms3d.euler.euler2mat(*rotation_euler)
rotation_vector = rotation_matrix[:, :2].T.ravel(
).tolist()  # Take first two columns

# Hardcoded joint positions (example values for 16 DOFs)
joint_pkg = np.load('/home/haozhe/Desktop/RoboVerse/_data/allegro/box_allegro_batched_output.npy') # Replace with desired joint angles



joint_positions=joint_pkg[40]


joint_positions=normalize_j0_joints(joint_positions, factor=2) # IN -180 TO 180
joint_positions=re_direct_j2_thj(joint_positions)
joint_positions=norm_th0(joint_positions,scale=2)

joint_positions=joint_positions.tolist()


print(joint_positions)
# joint_positions = torch.tensor(np.array(joint_positions), dtype=torch.float, device=device)
# Combine translation, rotation, and joint positions into hand pose
hand_pose = torch.tensor(
    translation + rotation_vector + joint_positions,
    dtype=torch.float,
    device=device
).unsqueeze(0)

# Set the parameters for the hand model
hand_model.set_parameters(hand_pose)

link_directions = {
    "link_3.0_tip": {
        "directions": [
            [1.0, 0.0, 0.0],
        ],
        "color": [255, 0, 0, 255],
        "arrow_scale": 0.02,
    },
    "link_7.0_tip": {
        "directions": [
            [1.0, 0.0, 0.0],
        ],
        "color": [255, 0, 0, 255],
        "arrow_scale": 0.02,
    },
    "link_11.0_tip": {
        "directions": [
            [1.0, 0.0, 0.0],
        ],
        "color": [255, 0, 0, 255],
        "arrow_scale": 0.02,
    },
    "link_15.0_tip": {
        "directions": [
            [1.0, 0.0, 0.0],
        ],
        "color": [255, 0, 0, 255],
        "arrow_scale": 0.02,
    },
}

# Generate the hand mesh
hand_mesh = hand_model.get_trimesh_data(0, link_directions=link_directions)

# Visualize the hand mesh
hand_mesh.show()

[0.20470209419727325, 0.7585015892982483, 0.6658748984336853, 0.14746171236038208, 0.14579585194587708, 1.187990665435791, 0.6308642029762268, 0.33612382411956787, -0.23437094688415527, 1.2533916234970093, 0.5505273938179016, 0.5851162075996399, 1.8252416849136353, 0.770115077495575, 0.7487932443618774, 0.36217349767684937]
